# STATS 507 Practice Exam

**name:** _______________________________

**uniqname:** ___________________________

You have 120 minutes to complete this exam.

This is an "open book" exam: you may refer to the course lecture slides, your past homework submissions, the textbook, stackoverflow, Google/Baidu, and other web pages about Python.

You do not have to perform "error checking" unless you are explicitly asked to do so.

You may not communicate with anyone (other the course instructors) during the exam. The code you submit must be yours alone.

You must stay connected to the class Zoom session with your camera on during the exam.

## Academic Integrity

Communicating with anyone other than the course instructors during the exam is strictly forbidden.
You may not share this exam or your solutions with your others (even after the exam is over).
You may not accept information about the exam from your classmates.

__Type your full name in the cell below to certify that you will follow this policy.__

-------->  Your Name Here <------------

In [1]:
import re
import math
import itertools
import functools
import numpy as np
import scipy as sp
import pandas as pd
import sqlite3
import torch

### Problem 1

Write a function that computes the circumference of a circle given its radius.

In [11]:
def get_circum(radius):
    # YOUR CODE HERE
    return 2 * math.pi * radius

In [12]:
assert abs(get_circum(5) - 31.41) < 1e-2

### Problem 2

Suppose you have a list that contains the all but one of the first $N$ natural numbers (including zero). Which one is missing? (Below assume $N$ is len(arr) + 1.)

In [13]:
def missing_number(arr):
    # YOUR CODE HERE
    arr.sort()
    for i in range(len(arr)):
        if i != arr[i]:
            return i
    return len(arr)

In [14]:
assert missing_number([0, 1, 9, 4, 5, 2, 3, 8, 7]) == 6
assert missing_number([0, 1, 4, 2, 5]) == 3
assert missing_number([0, 4, 2, 3, 1]) == 5

### Problem 3
Write a function called `nested_dictionary`, which takes a list of strings as its only argument, converts the list into a "nested dictionary of keys" (see test cases), and returns the dictionary. Your function should raise a `TypeError` in the event that the input is not a list and a `ValueError` if the input list is not a list of strings.

In [15]:
def nested_dictionary(lst):
    # YOUR CODE HERE
    if not isinstance(lst, list):
        raise TypeError("input isn't a list")
    for x in lst:
        if not isinstance(x, str):
            raise ValueError("element isn't a string")
    ret = {}
    head = ret
    for x in lst:
        head[x] = {}
        head = head[x]
    return ret

In [16]:
assert nested_dictionary([]) == {}
assert nested_dictionary(["hotdog"]) == {"hotdog": {}}
assert nested_dictionary(["a", "b", "c"]) == {"a": {"b": {"c": {}}}}

In [17]:
try:
    nested_dictionary(["a", "b", 42, "c"])
    assert False
except ValueError as e:
    pass
    
try:
    nested_dictionary("sweet potato")
    assert False
except TypeError as e:
    pass

### Problem 4
Write a class named `Vehicle` with a constructor that takes two arguments: `max_gallons` and `mpg`. The former is the vehicle's fuel tank capacity and the latter is the number of miles the vehicle can travel on each gallon of gas. Implement a method called `get_range` that returns the number of miles the vehicle can travel on a full tank of gas.

In [20]:
class Vehicle:
    # YOUR CODE HERE
    def __init__(self, max_gallons, mpg):
        self.max_gallons = max_gallons
        self.mpg = mpg
    
    def get_range(self):
        return self.max_gallons * self.mpg

In [21]:
# A Prius gets 54 mpg and its gas tank holds 11.4 gallons of gas
prius = Vehicle(11.4, 54)
assert abs(prius.get_range() - 615.6) < 1e-8

### Problem 5
For this problem, a "bigram" is a pair of consecutive characters in a string, where both characters are letters. For example, the string `"hi bobo"` contains exactly three unique bigrams: `"hi"`, `"bo"`, and `"ob"`. The second bigram, `"bo"`, appears twice, while the others appear once.

Write a function that takes a string as input and returns the most frequently occurring bigram in this string. You may assume that the input string contains only lowercase letters, spaces, and punctuation. You may also assume that the most common bigram is unique.

In [22]:
def commonest_bigram(x):
    # YOUR CODE HERE
    found = {}
    letters = 'abcdefghijklmnopqrstuvwxyz'
    for i in range(len(x) - 1):
        bi = x[i:(i+2)]
        if bi[0] in letters and bi[1] in letters:
            if bi in found:
                found[bi] += 1
            else:
                found[bi] = 1
    longest = max(found.values())
    for k, v in found.items():
        if v == longest:
            return k

In [23]:
assert commonest_bigram("hi bobo") == "bo"
assert commonest_bigram("hey hey, help is on the way!") == "he"
assert commonest_bigram("fun run. in the sun!") == "un"

### Problem 6

Write a function that validates credit card numbers from a particular bank. A string is a valid credit card number for this bank if and only if the following conditions hold:
- It must start with a 2, 4, or 6.
- It must contain exactly 16 digits.
- It must only consist of digits (0-9) and hyphens.
- If it contains hyphens, its digits are separate by hyphens into groups of four.
- It must NOT use any other separator like space or underscore.

In [25]:
def valid_cc(cc_number):
    # YOUR CODE HERE
    return re.match(r"[246]\d\d\d(-\d\d\d\d){3}$|[246]\d{15}$", cc_number)

In [26]:
assert valid_cc("2445571712286433")
assert valid_cc("6291002357881884")
assert valid_cc("4113-7221-8881-4335")

assert not valid_cc("62911123578818840")      # 17 digits in card number
assert not valid_cc("4113-7221-8881 - 4335")  # Separators other than '-' are used
assert not valid_cc("6291112357x881884")      # Contains non-digit characters
assert not valid_cc("5291112357881884")       # Doesn't start with 2, 4, or 6
assert not valid_cc("4113-7221-8881")         # 12 digits in card number

### Problem 7
The density of a multivariate normal distribution is
$f(x) = (2\pi)^{-k/2}\mathrm{det}(\Sigma)^{-1/2}e^{-\frac{1}{2}(x - \mu)^\top \Sigma^{-1} (x - \mu)}$, where $x$ is the data, $k$ is the dimension of the data, $\mu$ is the mean, and $\Sigma$ is the covariance.

Write a function that computes this density using only numpy (and built-in Python commands);
do not use `scipy.stats.multivariate_normal`.

In [27]:
def mvn_pdf(x, mean, cov):
    # YOUR CODE HERE
    ret = (2* np.pi)**(-.5 * len(x)) * np.linalg.det(cov)**(-0.5)
    sqdist = (x - mean).dot(np.linalg.inv(cov)).dot(x-mean)
    ret *= np.exp(-.5 * sqdist)
    return ret

In [28]:
from scipy.stats import multivariate_normal

x = np.zeros(2)
mean = np.zeros(2)
cov = np.eye(2)
claimed = mvn_pdf(x, mean, cov)
rv = multivariate_normal(mean=mean, cov=cov)
correct = rv.pdf(x)
assert (claimed - correct) < 1e-8

x = np.ones(2)
mean = np.ones(2) * .7
mean[1] += .3
cov = np.eye(2) + 0.5
cov[0, 0] += 3
claimed = mvn_pdf(x, mean, cov)
rv = multivariate_normal(mean=mean, cov=cov)
correct = rv.pdf(x)
assert (claimed - correct) < 1e-8

### Problem 8
A number is said to be *enchanted* if the sum of its digits raised to their respective positions is the number itself.

Write a function that determines whether a number is enchanted.

Note: Position of the digit is 1-indexed.

In [29]:
def is_enchanted(n):
    # YOUR CODE HERE
    r = 0
    for i in range(len(str(n))):
        r += int(str(n)[i]) ** (i + 1)
    return r == n

In [30]:
# 7^1 + 5^2 = 7 + 25 = 32
assert not is_enchanted(75)

# 1^1 + 3^2 + 5^3 = 1 + 9 + 125 = 135
assert is_enchanted(135)

# 8^1 = 8
assert is_enchanted(8)

### Problem 9

You and your friend Charles are playing a game of chess. You've both been having trouble following the rules (especially Charles) and now your chess board may be in an invalid state. One way a board can be invalid is if two kings appear next to each other. (A square is "next to" as many as eight other squares: left/right/top/bottom as well as above-left/above-right/below-left/below-right.)

Write a function that takes an $8\times 8$ numpy array. This array represents a chess board. The array is all zeros except for two ones that indicate the locations of the kings. Return `False` if and only if the two kings are next to each other. 

In [31]:
def valid_board(board):
    # YOUR CODE HERE
    for i in range(7):
        for j in range(7):
            if board[i:(i+2),j:(j+2)].sum() > 1.5:
                return False
    return True

In [32]:
board1 = np.zeros((8, 8))
board1[1,1] = 1
board1[6,5] = 1
assert valid_board(board1)

board2 = np.zeros((8, 8))
board2[2,2] = 1
board2[2,3] = 1
assert not valid_board(board2)

board3 = np.zeros((8, 8))
board3[2,2] = 1
board3[3,3] = 1
assert not valid_board(board3)

### Problem 10
Write a function which generates a square "dartboard" of side length $n$, as shown in the test cases below. On the dartboard, a number displayed increases the closer its position is to the center. You may assume $n < 19$.

In [33]:
def create_dartboard(n):
    # YOUR CODE HERE
    row, top = 0, []
    for i in range(n, 0, -2):
        row += int('{:0^{}}'.format('1'*i, n))
        top.append(row)
    bottom = top[::-1][1:] if n%2 else top[::-1]
    return top+bottom

In [34]:
assert create_dartboard(3) == [
    111,
    121,
    111
]

assert create_dartboard(4) == [
    1111,
    1221,
    1221,
    1111
]

assert create_dartboard(9) == [
    111111111,
    122222221,
    123333321,
    123444321,
    123454321,
    123444321,
    123333321,
    122222221,
    111111111
]

### Problem 11

Using PyTorch and gradient descent, find $\hat \beta_0$ and $\hat \beta_1$ that minimizes the sum of squared errors: $\sum_{i = 1}^N (\hat y_i - y)^2$. Here, for $i=1,\ldots,N$, $x_i$ is a univariate predictor, $y_i$ is a univariate response, and $\hat y_i = \hat \beta_0 + \hat \beta_1 x_i$. The function you write takes six arguments: `x` (the predictors), `y` (the response), `init_betahat0` (a starting value for $\hat \beta_0$), `init_betahat1` (a starting value for $\hat \beta_1$), `num_iters` (the total number of gradient descent iterations), and `lr` (the learning rate).

In [36]:
def ols_via_gd(x, y, init_betahat0, init_betahat1, num_iters, lr):
    # YOUR CODE HERE
    betahat0 = torch.tensor(init_betahat0, requires_grad=True)
    betahat1 = torch.tensor(init_betahat1, requires_grad=True)
    
    for i in range(num_iters):
        yhat = betahat0 + betahat1 * x
        sse = ((yhat - y)**2).sum()
        sse.backward()
        with torch.no_grad():
            betahat0 -= lr * betahat0.grad
            betahat1 -= lr * betahat1.grad
            betahat0.grad.zero_()
            betahat1.grad.zero_()
    
    return betahat0, betahat1

In [37]:
x = torch.arange(10)
y = 3 * x + 2
betahat0, betahat1 = ols_via_gd(x, y, 0.0, 0.0, 1000, 1e-3)
assert abs(betahat0 - 2) < 0.1
assert abs(betahat1 - 3) < 0.1

### Problem 12
Write a function that takes takes two arguments: the filename of an SQLite database and a stock ticker symbol.
The SQLite database has a table called "stocks" that lists daily closing prices for stocks. It has three columns: date, symbol, and price. Your function should return the average price for the stock with the specified symbol.

In [38]:
def average_price(db_file, ticker_symbol):
    with sqlite3.connect(db_file) as conn:
        # YOUR CODE HERE
        c = conn.cursor()
        q = 'select avg(price) from stocks where symbol=?'
        for row in c.execute(q, (ticker_symbol,)):
            return row[0]

In [39]:
claimed = average_price("stocks.db", "APPL")
assert abs(claimed - 116.4) < 1e-8
